# Initializer of the AARMP model in the following steps

1. #### Download and of NHD Data for Valley Bottom Models and creation of the two valley bottom models

	This checks for the existence of the data and then attempts download the NHD-HR data from Amazon S3 servers and create the remaining rasters variables from it. The some functions involved in this prep must be done within a python environment with access to the arcpy module.  

1. #### Download and inital prep of Landsat Data

	This checks for the existence of the final output and then, if it doesn't exist commences looking for the principal dataset and subsequent derivatives.

1. #### Georeference Landsat Data

	Begins creating the Valley Bottom raster based on the inputs prepared in step 1. 

1. #### Calculate Landsat NDWI and NDSI Indicies
    
    Calculation of Indicies derived from landsat bands (soil and water)

1. #### Initiation of the Landcover Analysis

	Begins process of segmenting NAIP Images, creating raster stacks of training data, creating landcover classifications, and lastly creating the riparian classification based on the landcover classes.

Preliminary necessary variabels

In [8]:
from shapely.geometry import Point
from shapely.ops import linemerge
import rasterio as rio
import pandas as pd
import geopandas as gpd
import gdal, osr
import numpy as np
import os, shutil
import math
from rasterio.merge import merge as merge_tool

import Utilities as utils
import ValleyBottomRastersPrep
import VBET_ValleyBottomModel
import RSAC_RegressionModel
import getLandsatData
import buildLandsat
import RasterCalculations as rs
from createRiparianClassifications import createClassification

import logging

# Set to either INFO or DEBUG
logging.basicConfig(level=logging.INFO)

data_dir = os.path.abspath(r"M:\Data")
logging.info("Using main data directory '%s' for all files" % data_dir)


# Initialize Data Directory Structure
utils.initializeDirectoryStructure(data_dir)

# Input vector files directory and files
#vectors_dir = os.path.join(data_dir, "initial_model_inputs")

vb_classification_pnts = os.path.join(utils.inputs_dir, "VM_TrainingData_20180619.shp")
watersheds_file = os.path.join(utils.inputs_dir, "WBDHU4_Arizona.shp")
# Specify the Area of interest file for landsat downloads and landsat path/row file (wrs2). An intersect will find all necessary path/rows
aoi_file = os.path.join(utils.utils.inputs_dir, "NAIP_AZ_Boundary.gpkg")
landsat_wrs2_file = os.path.join(utils.inputs_dir, "wrs2_descending.shp")
naip_acqui_vector = os.path.join(utils.inputs_dir, "NAIP2015_AcquisitionDates_Dissolve.gpkg")
# Find and download the NAIP acquisition dates for year of naip imagery. USDA has this,
# but it's hard to get. 2023 features in the layer with ~120 feature limit per download, 
# so had to chunk it into 100s for export then merge back together. Could also script here. 
# Source: https://gis.apfo.usda.gov/arcgis/rest/services/NAIP_Status/NAIP_Image_Dates/MapServer/52
naip_acqui_raster = os.path.join(utils.inputs_dir, "NAIP2015_AcquisitionDates_Dissolve.tif")
getLandsatData.rasterizeVector(naip_acqui_vector, naip_acqui_raster)

INFO:root:Using main data directory M:\Data for all non-NAIP files


### 1. Download and of NHD Data for Valley Bottom Models and creation of the two valley bottom models

In [ ]:
# Create Watersheds Data Directory
watersheds_dir = os.path.join(utils.valley_bottom_dir, "Watersheds")

# ------------------------------------------------
# Download of necessary NHD raster data
utils.getNHDData(utils.valley_bottom_dir)
  
# ------------------------------------------------
# VBET-based Valley Bottom Model, Currently the model used
"""Large Slope Threshold: The value that represents the upper limit of slopes that will be included in the valley bottom
for the 'large' portions of the network."""
large_slope_thresh = 3.2
"""Medium Slope Threshold: The value that represents the upper limit of slopes that will be included in the valley bottom
 for the 'medium' portions of the network."""
medium_slope_thresh = 4.5
"""Small Slope Threshold: The value that represents the upper limit of slopes that will be included in the valley bottoms
 for the "small" portions of the network."""
small_slope_thresh = 22

slope_thresholds = {"Small": small_slope_thresh, "Medium": medium_slope_thresh, "Large": large_slope_thresh}

"""High Drainage Area Threshold: The drainage area value in square meters. Streams whose upstream drainage area is greater
than this value will be considered the "large" portion of the network, and whose maximum valley bottom width will be
represented with the "Large Buffer Size" parameter."""
high_drainage_area_thresh = 1000000  # (m2)
"""Low Drainage Area Threshold: The drainage area value in square meters. Streams whose upstream drainage area is less
than this value will be considered the "small" portion of the network, and whose maximum valley bottom width will be
represented with the "Small Buffer Size" parameter. Streams whose upstream drainage area is between the high and low
drainage area thresholds will be considered the "medium" portion of the network and their maximum valley bottom width
represented by the "Medium Buffer Width" parameter."""
low_drainage_area_thresh = 40000  # (m2)

drainage_area_thresh = {"High": high_drainage_area_thresh, "Low": low_drainage_area_thresh}

VBET_ValleyBottomModel.createVBETValleyBottom(utils.valley_bottom_dir, watersheds_dir, slope_thresh_dict, overwrite=False, cleanup=False)

# ------------------------------------------------
# RSAC Regression Model - better potential, currently poor in lower, flatter areas
#RSAC_RegressionModel.valleyBottomRegression(watersheds_file, vb_classification_pnts, nhd_dir, watersheds_dir)

### 2. Download and inital preperationof Landsat Data


In [ ]:
# Only DOY calc to determine which landsat scenes to download. Make sure it's the same as the NAIP year
year = 2015
# Maximum clouds over landsat scene for download on date
cloud_limit = 40

# Create landsat folder for Top of Atmosphere Corrections
toa_dir = os.path.join(utils.base_landsatdir, "TOA_Corrected")
utils.useDirectory(toa_dir)


# ------------------------------------------------
# Download landsat and run TOA calculation and cloud mask
getLandsatData.getLandsatTOA(utils.base_landsatdir, naip_acqui_raster, aoi_file, landsat_wrs2_file, cloud_limit)

# ------------------------------------------------
# Using the NAIP DOY acquisition try to match for the landsat values at that location 
buildLandsat.createLandatDOYCompositeForAOI(naip_acqui_raster, data_dir, toa_dir, utils.base_landsatdir,
                                   landsat_toa_naipdate_merge, ls_nodata=landsat_nodata)

### 3. Georeference Landsat Data

The Landsat data is spatially offset from the NAIP data and needs to be georeferenced to overlap the best we can. The Easiest way to do this is in Arc, but you can only georeference 3 bands at a time. Workflow used was create a single set of georeferencing points and then to georeference bands 1-3, then 2-6, 7, then 8, and then use Composite Bands tool to merge them all back together. There's no easy way to script this without arcpy.

Provided is a sample set of control points in a text file that can be used

In [ ]:
georef_cntrl_points = os.path.abspath(vectors_dir, "ControlPoints_20180529.txt")
final_georef_landsat_composite = os.path.join(utils.base_landsatdir, "Landsat1to8_TOA_NAIPAcquiDate_merge_rectified.tif")

### 4. Calculate Landsat NDWI and NDSI Indicies

In [ ]:
rs.calculateLandsatIndicies(final_georef_landsat_composite, ndwi_outdir=utils.ndwi_qquad_dir, ndsi_outdir=utils.ndsi_qquad_dir)

### 5. Initiation of the Landcover Analysis

In [ ]:
# In acres, the size of area which will be assessed for density of vegetation classes
vegetation_assessment_area = 0.1

rf_args = {"maxdepth": 250,
           "n_est": 100,
           "n_job": 2,
           "min_per_leaf": 50,
           "crit": "entropy"}  # gini or entropy}

riplims = {"xero_limit": "StdDev",
           "meso_limit": 0.7,
           "hydro_limit": 0.9}


base_data_dir = os.path.abspath(r"../Data")

area_of_interest = gpd.read_file(os.path.abspath(r"M:\Data\initial_model_inputs\Ecoregions_AOI.gpkg"))

createClassification(area_of_interest,
                     dataDir=base_data_dir,
                     vaa=vegetation_assessment_area,
                     classifier_args=rf_args,
                     riparian_lims=riplims)